In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics supervision opencv-python-headless numpy matplotlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 14.5 MB/s eta 0:00:00


In [9]:
# Chemins des fichiers
video_path = "/content/drive/MyDrive/runYOLO/testing/Video4.mp4"
model_path = "/content/drive/MyDrive/runYOLO/best.pt"
output_path = "/content/drive/MyDrive/runYOLO/output/Video4.mp4"

In [10]:
from ultralytics import YOLO
import cv2
import numpy as np
from collections import deque
import supervision as sv

class BallAnnotator:
    def __init__(self, radius: int, buffer_size: int = 5, thickness: int = 2):
        self.color_palette = sv.ColorPalette.from_matplotlib('jet', buffer_size)
        self.buffer = deque(maxlen=buffer_size)
        self.radius = radius
        self.thickness = thickness

    def interpolate_radius(self, i: int, max_i: int) -> int:
        if max_i == 1:
            return self.radius
        return int(1 + i * (self.radius - 1) / (max_i - 1))

    def annotate(self, frame: np.ndarray, detections: sv.Detections) -> np.ndarray:
        xy = detections.xyxy[:, :2].astype(int)
        self.buffer.append(xy)
        for i, xy in enumerate(self.buffer):
            color = self.color_palette.by_idx(i)
            interpolated_radius = self.interpolate_radius(i, len(self.buffer))
            for center in xy:
                frame = cv2.circle(
                    img=frame,
                    center=tuple(center),
                    radius=interpolated_radius,
                    color=color.as_bgr(),
                    thickness=self.thickness
                )
        return frame


class BallTracker:
    def __init__(self, buffer_size: int = 10):
        self.buffer = deque(maxlen=buffer_size)

    def update(self, detections: sv.Detections) -> sv.Detections:
        xy = detections.xyxy[:, :2]
        self.buffer.append(xy)

        if len(detections.xyxy) == 0:
            return detections

        centroid = np.mean(np.vstack(self.buffer), axis=0)
        distances = np.linalg.norm(xy - centroid, axis=1)
        index = np.argmin(distances)
        return sv.Detections(xyxy=np.array([detections.xyxy[index]]))


# Charger le modèle YOLO
model = YOLO(model_path)

# Lire la vidéo
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Initialiser les classes
ball_annotator = BallAnnotator(radius=15, buffer_size=10)
ball_tracker = BallTracker(buffer_size=10)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Détection avec YOLO
    results = model.predict(frame, save=False, conf=0.5)

    # Extraire les coordonnées des boîtes englobantes
    boxes = results[0].boxes
    if len(boxes) > 0:
        xyxy = np.array([box.xyxy[0].tolist() for box in boxes])  # Convertir en format (x1, y1, x2, y2)
        detections = sv.Detections(xyxy=xyxy)
    else:
        detections = sv.Detections(xyxy=np.empty((0, 4)))  # Aucune détection

    # Mise à jour du tracker
    tracked_detections = ball_tracker.update(detections)

    # Annotation de la trajectoire
    annotated_frame = ball_annotator.annotate(frame, tracked_detections)

    # Sauvegarder la vidéo
    out.write(annotated_frame)

# Libérer les ressources
cap.release()
out.release()
print(f"Vidéo sauvegardée à : {output_path}")



0: 384x640 (no detections), 130.6ms
Speed: 17.7ms preprocess, 130.6ms inference, 113.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.8ms
Speed: 2.4ms preprocess, 14.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 12.2ms
Speed: 2.5ms preprocess, 12.2ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.3ms
Speed: 4.2ms preprocess, 14.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 20.0ms
Speed: 2.4ms preprocess, 20.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 18.3ms
Speed: 1.9ms preprocess, 18.3ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.0ms
Speed: 1.9ms preprocess, 14.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 14.7ms
Speed: 1.9ms preprocess, 14.

In [11]:
from IPython.display import HTML
from base64 import b64encode

# Lecture de la vidéo sauvegardée
def display_video(path):
    mp4 = open(path, 'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f'<video width=800 controls><source src="{data_url}" type="video/mp4"></video>')

display_video(output_path)


Output hidden; open in https://colab.research.google.com to view.